# Install detectron2

In [ ]:
# install dependencies:
!pip install pyyaml==5.1
!pip install --upgrade wandb
!pip install torch==1.7.1
!pip install torchvision==0.8.2
!wandb login 1c88ad7e526f7502ed1e006284699744e996f134
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 8.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=ce99bac4e084398ac8ac16d9f036d388e26467f340ddad1b31648e024603ef15
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 2.1MB 9.0MB/s 
     |████████████████████████████████| 133kB 32.2MB/s 
     |████████████████████████████████| 163kB 28.0MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=65a4273e7a58fac37ad3cd56f1911207f6fd08955f77d8f418167f52ecfdf6ef
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad1

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.0MB 1.9MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=3618abee3a2543a00b18122fe14a884b18dec38208d506acdcfd532f861cf69c
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore


In [ ]:
# Some basic setup:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import zipfile
from google.colab import drive
drive.mount("/content/gdrive")

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog



Mounted at /content/gdrive


# Train on a custom dataset

## Prepare the dataset

In [ ]:
zip_ref = zipfile.ZipFile("/content/gdrive/My Drive/gestureData-COCO.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Register the gestures dataset to detectron2


In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("gestures_train", {}, "/tmp/gestureData-COCO/train/trainCOCO.json", "/tmp/gestureData-COCO/train")
register_coco_instances("gestures_val", {}, "/tmp/gestureData-COCO/val/valCOCO.json", "/tmp/gestureData-COCO/val")


In [ ]:
print(MetadataCatalog.get("gestures_train"))

## Set up Weights & Biases

In [ ]:
# Initialize Weights & Biases
import wandb
wandb.init(project="GestureRecognition", 
           sync_tensorboard=True)



wandb: Currently logged in as: leopaz (use `wandb login --relogin` to force relogin)


## Train!

Now, let's fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the gesture dataset


In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("gestures_train",)
cfg.DATASETS.TEST = ("gestures_val",)
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.0050
cfg.SOLVER.MAX_ITER = 2500 # 5000 x 8 = 40,000 => 3,000 instances so rought 13 epochs
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # regions of proposal
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10 # number of classes to train

cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[04/14 23:58:12 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:04, 38.3MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due 

[04/14 23:58:19 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/masks.py:348: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()


[04/14 23:58:43 d2.utils.events]:  eta: 0:47:38  iter: 19  total_loss: 3.07  loss_cls: 2.214  loss_box_reg: 0.1354  loss_mask: 0.6911  loss_rpn_cls: 0.01598  loss_rpn_loc: 0.008468  time: 1.1520  data_time: 0.1337  lr: 9.9905e-05  max_mem: 6243M
[04/14 23:59:06 d2.utils.events]:  eta: 0:47:24  iter: 39  total_loss: 1.084  loss_cls: 0.2316  loss_box_reg: 0.1429  loss_mask: 0.6842  loss_rpn_cls: 0.01592  loss_rpn_loc: 0.009045  time: 1.1554  data_time: 0.1288  lr: 0.0001998  max_mem: 6244M
[04/14 23:59:29 d2.utils.events]:  eta: 0:47:06  iter: 59  total_loss: 0.9987  loss_cls: 0.1822  loss_box_reg: 0.1459  loss_mask: 0.6491  loss_rpn_cls: 0.002976  loss_rpn_loc: 0.006213  time: 1.1603  data_time: 0.1289  lr: 0.0002997  max_mem: 6244M
[04/14 23:59:53 d2.utils.events]:  eta: 0:46:51  iter: 79  total_loss: 0.9286  loss_cls: 0.1723  loss_box_reg: 0.1773  loss_mask: 0.572  loss_rpn_cls: 0.001442  loss_rpn_loc: 0.004069  time: 1.1640  data_time: 0.1303  lr: 0.00039961  max_mem: 6244M
[04/15 00

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

Launching TensorBoard...

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
cfg.MODEL.DEVICE = "cpu";
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
path = "/tmp/gestureData-COCO/train/"
images = ["frame_03_01_0001", "frame_03_02_0026", "frame_08_08_0059", "frame_03_06_0023", "frame_09_03_0119", "frame_09_04_0106", "frame_08_05_0010", "frame_03_07_0088", "frame_08_10_0031", "frame_09_10_0198", "frame_03_05_0057", "frame_03_08_0048", "frame_08_04_0025", "frame_09_05_0185"]
for image in images:
  im = cv2.imread(path + image + ".png")
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  v = Visualizer(im[:, :, ::-1],
        MetadataCatalog.get("gestures_train"),
        scale=0.5, 
        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
      )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

# Performance Metrics


In [ ]:
import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("gestures_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "gestures_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

# Export Model


In [ ]:
from google.colab import files
files.download('output/model_final.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>